In [1]:
import numpy as np
import pandas as pd

from libreco.data import DatasetPure
from libreco.algorithms import BPR, WideDeep
from libreco.evaluation import evaluate

from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


# Preprocess Data

In [2]:
books_df = pd.read_csv('../data/Books.csv', delimiter=';', low_memory=False)
ratings_df = pd.read_csv('../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../data/Users.csv', delimiter=';')

/var/folders/s0/4m_t_82936ggv17n0g0t7_dr0000gn/T/ipykernel_40313/4047143737.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../data/Users.csv', delimiter=';')


In [3]:
ratings_df.columns = ['user_id', 'item_id', 'label']

In [4]:
ratings_df = ratings_df[ratings_df['label'] != 0]

In [5]:
book_str_to_int = {book_id: i for i, book_id in enumerate(ratings_df['item_id'].unique())}
user_str_to_int = {user_id: i for i, user_id in enumerate(ratings_df['user_id'].unique())}

ratings_df['item'] = ratings_df['item_id'].map(book_str_to_int).to_numpy()
ratings_df['user'] = ratings_df['user_id'].map(user_str_to_int).to_numpy()

# Train / Eval Split

In [6]:
train_df, eval_df = train_test_split(ratings_df, test_size=0.2, random_state=0)

In [7]:
eval_user_mask = eval_df['user'].isin(train_df['user'].unique())
eval_item_mask = eval_df['item'].isin(train_df['item'].unique())

eval_df = eval_df[eval_user_mask & eval_item_mask]

# 'user', 'item' must be the first two columns of the dataframe
train_df = train_df[['user', 'item', 'label']]
eval_df = eval_df[['user', 'item', 'label']]

In [8]:
train_data, data_info = DatasetPure.build_trainset(train_df)
eval_data = DatasetPure.build_evalset(eval_df)

# Bayesian Personalized Ranking and Wide Deep


In [9]:
embedding_size = 16
n_epochs_bpr = 10
n_epochs_wide = 10
learning_rate = 0.01
batch_size = 256
random_seed = 42

In [10]:
# init BPR model
bpr = BPR(
    task="ranking",  
    data_info=data_info,
    embed_size=embedding_size,  
    n_epochs=n_epochs_bpr,  
    lr=learning_rate,  
    reg=None,  
    batch_size=batch_size,  
    num_neg=1,  
    use_tf=True,  
    seed=random_seed
)

In [11]:
# init the WideDeep model
wide_deep = WideDeep(
    task="ranking", 
    data_info=data_info,
    embed_size=embedding_size,
    n_epochs=n_epochs_wide,
    lr={'wide': learning_rate, 'deep': learning_rate/10},
    batch_size=batch_size,
    use_bn=True, 
    hidden_units=[64, 32],  
    reg=None,  
    dropout_rate=None,  
    num_neg=1,  
    seed=random_seed
)

In [12]:
# train BPR model
bpr.fit(
    train_data, 
    neg_sampling=True,
    verbose=2, 
    shuffle=True, 
    eval_data=eval_data, 
    metrics=["ndcg", "precision"]
)

Training start time: 2024-08-10 17:29:55


2024-08-10 17:29:55.888409: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-08-10 17:29:55.896630: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
train: 100%|██████████| 1356/1356 [00:10<00:00, 131.42it/s]


Epoch 1 elapsed: 10.339s
	 train_loss: 0.6237


eval_listwise: 100%|██████████| 16331/16331 [02:19<00:00, 117.14it/s]


	 eval ndcg@10: 0.0183
	 eval precision@10: 0.0037


...

	 eval ndcg@10: 0.0086
	 eval precision@10: 0.0018


train: 100%|██████████| 1356/1356 [00:09<00:00, 142.19it/s]


Epoch 10 elapsed: 9.540s
	 train_loss: 0.0419


eval_listwise: 100%|██████████| 16331/16331 [02:43<00:00, 99.64it/s] 


	 eval ndcg@10: 0.0082
	 eval precision@10: 0.0018


In [13]:
# train WideDeep model
wide_deep.fit(
    train_data, 
    neg_sampling=True,
    verbose=2, 
    shuffle=True, 
    eval_data=eval_data, 
    metrics=["ndcg", "precision"]
)

Training start time: 2024-08-10 17:46:55
Instructions for updating:
Colocations handled automatically by placer.


/Users/bromanchuk/.venv39/lib/python3.9/site-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
2024-08-10 17:46:55,902 - WARNING - From /Users/bromanchuk/.venv39/lib/python3.9/site-packages/keras/layers/normalization/batch_normalization.py:562: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
/Users/bromanchuk/.venv39/lib/python3.9/site-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Batc

total params: 7,663,453 | embedding params: 7,659,131 | network params: 4,322


train: 100%|██████████| 2711/2711 [00:21<00:00, 125.90it/s]


Epoch 1 elapsed: 21.534s
	 train_loss: 0.6343


...

	 eval ndcg@10: 0.0063
	 eval precision@10: 0.0013


train: 100%|██████████| 2711/2711 [00:21<00:00, 124.26it/s]


Epoch 10 elapsed: 21.818s
	 train_loss: 0.116


eval_listwise: 100%|██████████| 16331/16331 [11:20<00:00, 23.99it/s]


	 eval ndcg@10: 0.0070
	 eval precision@10: 0.0015


In [14]:
# evaluate the BPR model
bpr_eval_result = evaluate(bpr, eval_data, neg_sampling=True, metrics=["ndcg", "precision", "recall"])
print(f"Evaluation Results (BPR):\n{bpr_eval_result}")

eval_listwise: 100%|██████████| 16331/16331 [01:09<00:00, 235.18it/s]


Evaluation Results (BPR):
{'ndcg': 0.008218662924561774, 'precision': 0.001818627150817464, 'recall': 0.008496358810695952}


In [15]:
# evaluate the WideDeep model
wide_deep_eval_result = evaluate(wide_deep, eval_data, neg_sampling=True, metrics=["ndcg", "precision", "recall"])
print(f"Evaluation Results (WideDeep):\n{wide_deep_eval_result}")

eval_listwise: 100%|██████████| 16331/16331 [30:14<00:00,  9.00it/s]  


Evaluation Results (WideDeep):
{'ndcg': 0.00704205306980035, 'precision': 0.0014757210213704, 'recall': 0.0062481205328919365}


In [16]:
user_id = train_df.iloc[:10]['user'].values
item_id = train_df.iloc[:10]['item'].values

prediction = bpr.predict(user_id, item_id)
print(f"BPR Prediction for user {user_id} and item {item_id}: {prediction}")
prediction = wide_deep.predict(user_id, item_id)
print(f"WideDeep Prediction for user {user_id} and item {item_id}: {prediction}")

BPR Prediction for user [44828 13679 67574  5478 50595 44797  9787 67068    64 76536] and item [129010  52858 127059  27618 139955   4218  42336  43591    202   3399]: [0.998423   0.9999753  0.99999416 0.99969935 0.99830866 0.9891834
 0.99997854 0.9968299  0.9996921  0.99963665]
WideDeep Prediction for user [44828 13679 67574  5478 50595 44797  9787 67068    64 76536] and item [129010  52858 127059  27618 139955   4218  42336  43591    202   3399]: [0.9969753  0.99936503 0.96080226 0.99981624 0.9465156  0.9660195
 0.98954964 0.9977005  0.99968636 0.9982249 ]


In [17]:
train_df.iloc[:10]

,user,item,label
654197,44828,129010,8
203372,13679,52858,10
1000213,67574,127059,10
84497,5478,27618,8
740145,50595,139955,6
652558,44797,4218,5
145942,9787,42336,8
991352,67068,43591,8
413,64,202,10
1133263,76536,3399,7
